In [1]:
from src.rnn import ReccurentNeuralNetwork

In [2]:
class PerformanceEvaluation:
    def __init__(self, parameters : list[dict]) -> None:
        self.parameters = parameters
        self.evaluations = {}
        self.models = []

    def runEvaluations(self) -> None:
        for params in self.parameters :
            model = ReccurentNeuralNetwork(**params)
            model.prepareData()
            self.evaluations[model.modelName] = model.completeEvaluation()
    
    def chooseTopN(self, n : int = 3) -> dict:
        return self.evaluations

In [4]:
def getParamsRNN():
    params = []

    possibleStackSize = [1,2,3]
    possibleHiddenLayerSizes = [ [32], [64], [128],
                             [32, 32], [64, 64],
                             [64, 32], [128, 64],
                             [128, 64, 32] ]
    possibleHiddenStateSizes = [ 64, 128, 256 ]
    possibleBidirectionality = [True, False]
    
    for stackSize in possibleStackSize :
        for hiddenLayerSize in possibleHiddenLayerSizes :
            for hiddenStateSize in possibleHiddenStateSizes :
                for bidirectionality in possibleBidirectionality :
                    params.append({
                        "stackSize" : stackSize,
                        "hiddenLayers" : hiddenLayerSize,
                        "hiddenStateSize" : hiddenStateSize,
                        "bidirectionality" : bidirectionality,
                        "modelName" : f"EvaluationRNN-s={stackSize}_h={len(hiddenLayerSize)}_b={bidirectionality}_l={hiddenStateSize}",
                    })

    return params

In [5]:
ffnnEval = PerformanceEvaluation(getParamsRNN())
ffnnEval.runEvaluations()

Using cuda
Model Loaded from EvaluationRNN-s=1_h=1_b=True_l=64_n=1_h=64_bidirectional_nonLinearity=tanh_layers=1_l=32_EPOCHS=10_BATCH_SIZE=16.
|████████████████████████████████████████| 37/37 [100%] in 1.0s (38.39/s) 
{'Accuracy': 0.955015197568389, 'Precision': 0.9635123289756421, 'F1Score': 0.9588546376304936, 'Recall': 0.955015197568389}
Using cuda
Model Loaded from EvaluationRNN-s=1_h=1_b=False_l=64_n=1_h=64_nonLinearity=tanh_layers=1_l=32_EPOCHS=10_BATCH_SIZE=16.
|████████████████████████████████████████| 37/37 [100%] in 0.9s (40.99/s) 
{'Accuracy': 0.9291793313069909, 'Precision': 0.9411668849871399, 'F1Score': 0.9334189809360758, 'Recall': 0.9291793313069909}
Using cuda
Model Loaded from EvaluationRNN-s=1_h=1_b=True_l=128_n=1_h=128_bidirectional_nonLinearity=tanh_layers=1_l=32_EPOCHS=10_BATCH_SIZE=16.
|████████████████████████████████████████| 37/37 [100%] in 0.9s (39.55/s) 
{'Accuracy': 0.9249240121580548, 'Precision': 0.9515726488004322, 'F1Score': 0.9353653972707724, 'Recall'

In [6]:
metrics = {}
def getAccuracy():
    for model, evaluator in ffnnEval.evaluations.items() :
        thisEvaluator = evaluator['Test']
        metrics[model] = thisEvaluator.getAccuracy(), thisEvaluator.getPrecision(), thisEvaluator.getF1Score(average='macro'), thisEvaluator.getF1Score('micro'), thisEvaluator.getRecall()

In [20]:
def printAccuracy(metrics):
    # Sorting the dictionary
    metrics = dict(sorted(metrics.items(), key = lambda x : x[1], reverse = True))
    print(f"{'Model Name':50} | {'Accuracy':10.8} | {'Precision' :10.8} | {'F1-macro' :10.8} |  {'F1-micro' :10.8} | {'Recall' :10.8}")
    for model, metric in metrics.items() :
        print(f"{model:<50} | {metric[0]*100:<10.6}% | {metric[1]*100:<7.6}% | {metric[2]*100:<7.6}% | {metric[3]*100:<7.6}% | {metric[4]*100:<7.6}%")

In [8]:
getAccuracy()

In [21]:
printAccuracy(metrics)

Model Name                                         | Accuracy   | Precisio   | F1-macro   |  F1-micro   | Recall    
EvaluationRNN-s=3_h=1_b=True_l=64_n=3_h=64_bidirectional_nonLinearity=tanh_layers=1_l=64 | 98.7994   % | 98.836 % | 90.0127% | 98.7994% | 98.7994%
EvaluationRNN-s=2_h=2_b=True_l=64_n=2_h=64_bidirectional_nonLinearity=tanh_layers=2_l=128_l=64 | 98.7538   % | 98.7542% | 97.0912% | 98.7538% | 98.7538%
EvaluationRNN-s=2_h=2_b=True_l=128_n=2_h=128_bidirectional_nonLinearity=tanh_layers=2_l=128_l=64 | 98.7234   % | 98.7434% | 96.6337% | 98.7234% | 98.7234%
EvaluationRNN-s=3_h=2_b=True_l=64_n=3_h=64_bidirectional_nonLinearity=tanh_layers=2_l=64_l=64 | 98.7082   % | 98.7156% | 97.344 % | 98.7082% | 98.7082%
EvaluationRNN-s=2_h=2_b=True_l=64_n=2_h=64_bidirectional_nonLinearity=tanh_layers=2_l=64_l=64 | 98.693    % | 98.7206% | 89.6664% | 98.693 % | 98.693 %
EvaluationRNN-s=2_h=2_b=True_l=256_n=2_h=256_bidirectional_nonLinearity=tanh_layers=2_l=128_l=64 | 98.693    % | 98.7006% | 